In [55]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd


scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/media/workstation/Storage/GoogleProject/DeepLearningAlphaC.txt', scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open("TASK")
worksheet_list = spreadsheet.worksheets()
Accepted = spreadsheet.worksheet("Accepted").get_all_records()

In [56]:
# df = pd.DataFrame(Accepted).astype(str)

In [57]:
# StratCode = str(1190)
# u1 = df["Strategy"] == StratCode

# StratName = df.loc[u1].loc[0,"StratName"].replace("SQL", "LIVE")
# TimeFrame = df.loc[u1].loc[0,"TimeFrame"]
# asset_b = df.loc[u1].loc[0, "Asset_B"]
# asset_s = df.loc[u1].loc[0, "Asset_S"]
# if len(asset_b) >= len(asset_s):
#     RefInstrument = asset_b
# else:
#     RefInstrument = asset_s
# ConsecutiveSignalBars = df.loc[u1].loc[0, "HoldingPeriod"]
# OrderType = df.loc[u1].loc[0, "OrderType"]
# NumInstrument = len(RefInstrument.split(","))


In [59]:
def generate_cs(StratName, TimeFrame, StratCode, RefInstrument, ConsecutiveSignalBars, OrderType, NumInstrument):
    if TimeFrame == "M_60":
        fo = open("LIVETemplate_DeepLearning_Rotation_M60_R_LIVE.cs", "r+")
        line = fo.readlines()
        print("Using M60 Template!")
    elif TimeFrame in ["M_5", "M_15", "M_30"]:
        fo = open("LIVETemplate_DeepLearning_Rotation_M30ETC_R_LIVE.cs", "r+")
        line = fo.readlines()
        print("Using M30 Template!")

    for i, z in enumerate(line):
        if z.find("LIVETemplate_DeepLearning_Rotation_M30ETC_R_LIVE") != -1:
            line[i] = z.replace("LIVETemplate_DeepLearning_Rotation_M30ETC_R_LIVE", StratName)

        if z.find("LIVETemplate_DeepLearning_Rotation_M60_R_LIVE") != -1:
            line[i] = z.replace("LIVETemplate_DeepLearning_Rotation_M60_R_LIVE", StratName)

            
        if z.find("chg_strategycode") != -1:
            line[i] = z.replace("chg_strategycode", StratCode)

        if z.find("chg_refinstrument") != -1:
            line[i] = z.replace("chg_refinstrument", RefInstrument)
        
        if z.find("chg_consecutive") != -1:
            line[i] = z.replace("chg_consecutive", ConsecutiveSignalBars)
            
        if z.find("chg_order") != -1:
            line[i] = z.replace("chg_order", OrderType)

        if z.find("chg_numinstrument") != -1:
            line[i] = z.replace("chg_numinstrument", NumInstrument)

    f = open(r'Output_Live/{}.cs'.format(StratName), 'w')
    f.writelines(line)
    f.close()
    
def get_code(code):
    if code.find("_") != -1:
        return code.split("_")[1].lower()
    else:
        return code.lower()

In [62]:
# import os
# os.mkdir("Output_Live")

In [63]:
for d in Accepted:
    StratCode = str(d["Strategy"])
    StratName = str(d["StratName"]).replace("SQL", "LIVE")
    TimeFrame = str(d["TimeFrame"])
    asset_b = d["Asset_B"]
    asset_s = d["Asset_S"]
    if len(asset_b) >= len(asset_s):
        RefInstrument = asset_b
    else:
        RefInstrument = asset_s
    RefInstrument = RefInstrument.replace(" ", "")
    ConsecutiveSignalBars = str(d["HoldingPeriod"])
    OrderType = str(d["OrderType"])
    NumInstrument = str(len(RefInstrument.split(",")))
    
    print("=============")
    print(StratName, TimeFrame,ConsecutiveSignalBars, OrderType)
    print(RefInstrument)
    generate_cs(StratName, TimeFrame, StratCode, RefInstrument, ConsecutiveSignalBars, OrderType, NumInstrument)
    print("Done")
    print("=============")

_1190_RNNLongBreakout30Min_R_LIVE M_30 5 1
DIA,SMH,XBI
Using M30 Template!
Done
_1195_RNNBothBreakout60MinSharpe_R_LIVE M_60 10 0
AGG,EFA,EURUSD,IEF,TLT
Using M60 Template!
Done
_1197_RNNBothBreakout60MinProb_R_LIVE M_60 10 0
GLD,LQD,RSX,USO,XLV
Using M60 Template!
Done
_1198_RNNShortBreakout5MinSharpe_R_LIVE M_5 5 2
AUDUSD,NZDUSD,XAUUSD
Using M30 Template!
Done
_1199_RNNBothBreakout5MinSharpe_R_LIVE M_5 10 0
USDCAD
Using M30 Template!
Done
_1202_RNNProbProbW006BS_R_LIVE M_60 10 0
EWJ,EWW,TLT,USDCAD,USDJPY,VWO,XBI,XLK,XLY
Using M60 Template!
Done
_1202_2_RNNProbProbW006BS_R_LIVE M_60 8 0
EWG,EWJ,NZDUSD,TLT,VWO,XBI,XLE
Using M60 Template!
Done
_1203_RNNBreakoutReturnsY001_R_LIVE M_15 10 0
EEM,EMB,EURUSD,EWJ,GBPUSD,IBB,NZDUSD,UNG,USDCHF,XBI
Using M30 Template!
Done
_1203_2_RNNBreakoutReturnsY001_R_LIVE M_15 9 0
DBC,EMB,NZDUSD,XAUUSD,XBI
Using M30 Template!
Done
_1203_3_RNNBreakoutReturnsY001_R_LIVE M_15 9 0
EMB,NZDUSD,XBI,XLE
Using M30 Template!
Done
_1220_RNNBreakoutSharpeW008_R_LIVE M_

In [3]:
# for d in Accepted:
#     StratName = d["StratName"]
#     TimeFrame = d["TimeFrame"]
#     RefInstrument = d["Asset_B"] if d["Asset_B"] != "-" else d["Asset_S"]
#     dbTableLong = get_code(d["Code"]) + "b"
#     dbTableShort = get_code(d["Code"]) + "s"
#     ConsecutiveSignalBars = str(d["HoldingPeriod"])
#     OrderType = str(d["OrderType"])
#     print(StratName, TimeFrame, dbTableLong, dbTableShort, ConsecutiveSignalBars, OrderType)
#     print(RefInstrument)
#     print("=============")
#     generate_cs(StratName, TimeFrame, RefInstrument, dbTableLong, dbTableShort, ConsecutiveSignalBars, OrderType)


_1190_RNNLongBreakout30Min_R_SQL M_30 c001b c001s 5 1
DIA, SMH, XBI
Using M30 Template!
_1195_RNNBothBreakout60MinSharpe_R_SQL M_60 b001b b001s 10 0
AGG, EFA, EURUSD, IEF, TLT
Using M60 Template!
_1197_RNNBothBreakout60MinProb_R_SQL M_60 b003b b003s 10 0
GLD, LQD, RSX, USO, XLV
Using M60 Template!
_1198_RNNShortBreakout5MinSharpe_R_SQL M_5 c007b c007s 5 2
AUDUSD, NZDUSD, XAUUSD
Using M30 Template!
_1199_RNNBothBreakout5MinSharpe_R_SQL M_5 c010b c010s 10 0
USDCAD
Using M30 Template!
_1202_RNNProbProbW006BS_R_SQL M_60 w006b w006s 10 0
EWJ, EWW, TLT, USDCAD, USDJPY, VWO, XBI, XLK, XLY
Using M60 Template!
_1202_2_RNNProbProbW006BS_R_SQL M_60 w006b w006s 8 0
EWG, EWJ, NZDUSD, TLT, VWO, XBI, XLE
Using M60 Template!
_1203_RNNBreakoutReturnsY001_R_SQL M_15 y001b y001s 10 0
EEM, EMB, EURUSD, EWJ, GBPUSD, IBB, NZDUSD, UNG, USDCHF, XBI
Using M30 Template!
_1203_2_RNNBreakoutReturnsY001_R_SQL M_15 y001b y001s 9 0
DBC, EMB, NZDUSD, XAUUSD, XBI
Using M30 Template!
_1203_3_RNNBreakoutReturnsY001_R_SQ